<a href="https://colab.research.google.com/github/wubinstu/roop/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pre roop

!git clone https://github.com/wubinstu/roop.git
workdir = "/content/roop"
%pip install -r {workdir}/requirements.txt

!wget https://github.com/wubinstu/roop/releases/download/models/GFPGANv1.4.pth https://github.com/wubinstu/roop/releases/download/models/inswapper_128.onnx
!mv /content/GFPGANv1.4.pth /content/inswapper_128.onnx /content/roop/models/

source = f"{workdir}/media/*.jpg"
target = f"{workdir}/media/in.mp4"
output = f"{workdir}/media/out.mp4"

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# cp file
!cp -rf /content/drive/MyDrive/? {target}

In [ ]:
# starting

Device = "cuda" #@param ["cuda", "cpu"]
Processor = "face_swapper" #@param ["face_swapper face_enhancer", "face_swapper","face_enhancer"]
VideoEncoder = "libx264" #@param ['libx264', 'libx265', 'libvpx-vp9', 'h264_nvenc', 'hevc_nvenc']
VideoQuality = "35" #@param {type:"string"}
TempFormat = "jpg" #@param ["jpg", "png"]
TempQuality = "0" #@param {type:"string"}

KeepFPS = True #@param {type:"boolean"}
SkipAudio = False #@param {type:"boolean"}
KeepFrames = False #@param {type:"boolean"}
ManyFaces = False #@param {type:"boolean"}


KeepFPS ="--keep-fps" if KeepFPS==True else ""
SkipAudio ="--skip-audio" if SkipAudio==True else ""
KeepFrames ="--keep-frames" if KeepFrames==True else ""
ManyFaces ="--many-faces" if ManyFaces==True else ""



cmd = f"{workdir}/run.py --execution-provider {Device} -s {source} -t {target} -o {output} --frame-processor {Processor} --output-video-encoder {VideoEncoder} --output-video-quality {VideoQuality} {KeepFPS} {SkipAudio} {KeepFrames} {ManyFaces} --temp-frame-format {TempFormat} --temp-frame-quality {TempQuality}"
print("cmd:"+cmd)
!python $cmd
